# Imports

In [ ]:
import os
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt

In [ ]:
from Nim.Nim import Nim
from Nim.NimLogic import NimLogic

from Agents.MinimaxAgent import MinimaxAgent

Random seed for reproducibility

In [ ]:
np.random.seed(42)

# Agent Testing Function

In [ ]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):
    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = Nim(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        """ AGENT VALIDATION """
        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i])

        agent.compute_mean_nodes()

        mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, mean_nodes

# Test running function

In [ ]:
def run_tests(misere_agents, normal_agents, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    os.makedirs(save_path_prefix, exist_ok=True)

    print("-" * 60)
    classic_configuration = False

    if pile_count == 0 or max_pile == 0:
        classic_configuration = True
        initial_piles = np.tile([1, 3, 5, 7], (episodes, 1))
        print("Classic configuration used")
    else:
        initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))
        print(f"Configuration: pile_count: {pile_count}, max_pile: {max_pile}, max_depth: {max_depth}")

    print("-" * 60)
    misere_modes = np.random.choice([True, False], size=episodes)

    results = {}

    for agent_key in misere_agents.keys():
        misere_agent = misere_agents[agent_key]
        normal_agent = normal_agents[agent_key]
        results[agent_key] = test_agent(misere_agent, normal_agent, misere_modes, initial_piles)

    for agent_key, result in results.items():
        if not classic_configuration:
            filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-{pile_count}-{max_pile}.npz"
        else:
            filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-classic.npz"
        np.savez(filename, result)

# Plotting results

In [ ]:
def _first_existing_file(patterns):
    for p in patterns:
        if os.path.exists(p):
            return p
    return None

def plot_results(vary_parameter,
                 max_depth_values,
                 vary_values,
                 fixed_pile_count,
                 fixed_max_pile,
                 folder="savedData/Minimax",
                 save_folder="savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)

    # Updated to match the new agent key system
    models = ['n', 'c', 'p', 'a', 'cp', 'ca', 'pa', 'cpa']
    model_names = [
        'Normal',
        'Canonical',
        'P-pruning',
        'Aggressive',
        'Canonical + P-pruning',
        'Canonical + Aggressive',
        'P-pruning + Aggressive',
        'Canonical + P-pruning + Aggressive'
    ]

    # Extended color palette for 8 models
    colors = ['#4B0082', '#4169E1', '#9370DB', '#FF6347', '#32CD32', '#FFD700', '#FF1493', '#00CED1']

    metric_labels = [
        'Avg Nodes/Move',
        'Avg Nodes/Game',
        'Avg Moves/Game'
    ]

    # Adjusted bar width for more models
    bar_width = 0.08
    x_indices = np.arange(len(vary_values))

    for max_depth in max_depth_values:
        fig, axes = plt.subplots(1, 3, figsize=(15, 6), facecolor='white', sharex=True)
        axes = axes.ravel()

        all_metrics = {m: {i: [] for i in range(3)} for m in models}

        for vary_value in vary_values:
            for model in models:
                pile_count = vary_value if vary_parameter == "heap_count" else fixed_pile_count
                max_pile   = fixed_max_pile if vary_parameter == "heap_count" else vary_value

                candidate_files = [
                    f"{folder}/minimax-{model}-{max_depth}-{pile_count}-{max_pile}.npz",
                ]

                filename = _first_existing_file(candidate_files)

                if filename is None:
                    print(f"⚠️ Lipsă fișier pentru: depth={max_depth}, "
                          f"{vary_parameter}={vary_value} → {candidate_files[-1].split('/')[-1]}")
                    for i in range(3):
                        all_metrics[model][i].append(np.nan)
                    continue

                data                = np.load(filename)
                test_data           = data.get('arr_0')
                explored_nodes, moves_count, mean_nodes = test_data

                metrics_values = [
                    explored_nodes.sum() / moves_count.sum(),
                    explored_nodes.mean(),
                    moves_count.mean()
                ]
                for i in range(3):
                    all_metrics[model][i].append(metrics_values[i])

        for metric_idx, ax in enumerate(axes):
            for model_idx, model in enumerate(models):
                # Adjusted offset calculation for 8 models
                offset = -0.28 + 0.08 * model_idx
                values = all_metrics[model][metric_idx]
                bars = ax.bar(
                    x_indices + offset,
                    values,
                    width=bar_width,
                    color=colors[model_idx],
                    alpha=0.7,
                    label=model_names[model_idx]
                )

                ylim = ax.get_ylim()
                text_offset = 0.01 * ylim[1]  # un mic padding vertical

                for bar, val in zip(bars, values):
                    if not np.isnan(val):
                        ax.text(
                            bar.get_x() + bar.get_width() / 2,
                            bar.get_height() + text_offset,
                            f'{val:.1f}',
                            ha='center',
                            va='bottom',
                            fontsize=8,  # Reduced font size for readability
                            rotation=90,
                            bbox=dict(
                                facecolor='white',
                                edgecolor='none',
                                boxstyle='square,pad=0.1'
                            )
                        )

            ax.set_ylabel(metric_labels[metric_idx], fontsize=14)
            ax.set_xlabel(vary_parameter.upper(), fontsize=14)
            ax.set_xticks(x_indices)
            ax.set_xticklabels(vary_values)
            ax.grid(True, axis='y', alpha=0.3)
            if metric_idx == 0:
                # Position legend outside the plot area due to more models
                ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)

        plt.tight_layout(rect=[0, 0, 0.85, 0.97])  # Adjusted to accommodate legend
        save_path = os.path.join(save_folder, f"minimax_variation_{vary_parameter}_depth_{max_depth}.png")
        plt.savefig(save_path, facecolor='white', bbox_inches='tight')
        plt.close()

In [ ]:
EPISODES = 1000

PILE_COUNT = 4
MAX_PILE = 127

In [ ]:
for max_depth in [1, 2]:
    misere_agents = {
        'n': MinimaxAgent(misere=True, max_depth=max_depth),
        'c': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=True, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    normal_agents = {
        'n': MinimaxAgent(misere=False, max_depth=max_depth),
        'c': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=False, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    for pile_count in [2, 4, 8]:
        run_tests(misere_agents, normal_agents, max_depth, pile_count, MAX_PILE, EPISODES, "savedData/Minimax")

    for max_pile in [63, 255]:
        run_tests(misere_agents, *normal_agents, max_depth, PILE_COUNT, max_pile, EPISODES, "savedData/Minimax")

In [ ]:
plot_results(vary_parameter="heap_count", max_depth_values=[1, 2], vary_values=[2, 4, 8], fixed_pile_count=PILE_COUNT, fixed_max_pile=MAX_PILE)
plot_results(vary_parameter="max_heap", max_depth_values=[1, 2], vary_values=[63, 127, 255], fixed_pile_count=PILE_COUNT, fixed_max_pile=MAX_PILE)

# Max Depth testing on Smaller Configuration

In [ ]:
EPISODES = 1000

PILE_COUNT = 3
MAX_PILE = 5

MAX_DEPTH = 9999

In [ ]:
misere_agents = {
    'n': MinimaxAgent(misere=True, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

normal_agents = {
    'n': MinimaxAgent(misere=False, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

for pile_count in [2, 3, 4]:
    run_tests(*misere_agents, *normal_agents, MAX_DEPTH, pile_count, MAX_PILE, EPISODES, "savedData/Minimax")

for max_pile in [3, 7]:
    run_tests(*misere_agents, *normal_agents, MAX_DEPTH, PILE_COUNT, max_pile, EPISODES, "savedData/Minimax")

In [ ]:
plot_results(vary_parameter="heap_count", max_depth_values=[MAX_DEPTH], vary_values=[2, 3, 4], fixed_pile_count=PILE_COUNT, fixed_max_pile=MAX_PILE)
plot_results(vary_parameter="max_heap", max_depth_values=[MAX_DEPTH], vary_values=[3, 5, 7], fixed_pile_count=PILE_COUNT, fixed_max_pile=MAX_PILE)